# Import modules

In [ ]:
import torch

from model import FeNeC

from configs.config import config
from utils.loader import FeNeCDataLoader

# Choose the device

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    
print(f"Using device: {device}")

Using device: cuda


# Run the training and evalutating loop

For simplicity, you can also just pass extracted features from backbone with shapes as below to the model:
- X_train (num_train_samples, num_features)
- y_train (num_train_samples)
- X_test (num_test_samples, num_features)
- y_test (num_test_samples)

Here we are using our dataloader, but it requires the datasets to be stored in
given data directory and directory with dataset name (ex. "./data/tinyimagenet") and each task data need to be stored
in task_0.hdf5 files were 0 is number of task. In each of hdf5 files there are some keys specified. 
Look at utils/loader.py file for more information.

In [4]:
data_loader = FeNeCDataLoader(num_tasks=10, dataset_name="vit", load_covariances=True, load_prototypes=False, dataset_path="./data")
model = FeNeC(config, device=device)

accuracies = []

for i in range(10):
    X_train, y_train, X_test, y_test, covariances, prototypes = data_loader.get_data(i)
    
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    
    accuracy = (torch.sum((y_test.flatten().to(device) == predictions).int()) / X_test.shape[0]).item()
    accuracies.append(accuracy)

    print(f"Accuracy: {accuracy}")

print(f"Average incremental accuracy: {sum(accuracies)/len(accuracies)}")

Accuracy: 0.9820000529289246
Accuracy: 0.9510000348091125
Accuracy: 0.9263333082199097
Accuracy: 0.9077500700950623
Accuracy: 0.8965999484062195
Accuracy: 0.8861666321754456
Accuracy: 0.87628573179245
Accuracy: 0.8667500615119934
Accuracy: 0.8653333187103271
Accuracy: 0.8569999933242798
Average incremental accuracy: 0.9015219151973725
